# Steam Data Analysis

## Project Goals

<!-- PELICAN_BEGIN_SUMMARY -->

The motivation is gather, process and analyze Steam Store data to get insights about trends in the videogame market. As it is an online marketplace with public available data, it offers us more possibilities than analyzing console games data, where we would have to rely on an existing dataset.

We want to focus on two main aspects, first a general market analysis to know which genres are the most popular, pricing strategies and so on, which could be interesting for a new developer trying to make a new game or deciding a price policy. This has been studied already by other enthusiasts in internet, and also by Marketing companies helping publishers.

But to offer a different analysis, we want to also focus on the developers and publishers, to see which ones are the most successfull, how they have improved / worsen between the years, which titles have cemented their success and so on. In the light of recent years we have seen many acquisitions by large publishers such as Tencent, Microsoft and Sony, so this is very interesting concept.

This will be a complete data project, with a data acquisition section (by using some APIs and web scrapping), then data cleaning and joining data from different sources, an exploratory data analysis, and finally some key conclusions.


## Data Acquisition

This is the section where I struggled initially. There were several datasets available at [kaggle](https://www.kaggle.com/datasets), reddit and similar websites, but most were outdated or did not contain all the information I wanted to explore. Also I wanted to extract it directly from an API or use web scrapping, if possible, to learn a bit more (I already had experience with Twitter which has an excellent API).

[SteamSpy](https://steamspy.com/about) is a webpage which offers data about Steam games. In the past it was even able to deliver a good guess of sales, but that has become harder throughout the years. Check [VG Insights](https://vginsights.com/insights/article/how-to-estimate-steam-video-game-sales) for more information. It is also a good webpage if you want to explore market data on your own.
The most important thing for Steamspy is that it has its own API [here](https://steamspy.com/api.php). It can provide us easily with an already filtered list of games (not other apps or DLCs), and also some metrics not available at Steam directly such as an estimate of sales and the positive or negative reviews (Steam only gives us total number reviews).

Regarding Steam directly, the API is available at https://partner.steamgames.com/ , however you need a developer key and some (most of the functions) are tied to your key as they are intended to be used to manage your own products at the Steam store. Thanks to [Nik Davis](http://nik-davis.github.io) I discovered there were also a few API functions via the WEB API which can be used without a key at all. See here for more details: [StorefrontAPI](https://wiki.teamfortress.com/wiki/User:RJackson/StorefrontAPI).

Getting the information from Steam will be a bit more difficult, but it will give us additional metrics, such as release date, genre...

We will retrieve first the list of appids and the information available at Steam Spy, then get for each appid the information from Steam and combine them in an unique dataframe. There will be no loss of information as app ids are unique. Afterwards, we will perform cleaning and finally start analyzing our dataset.

## Process:

- Create an app list and gather available data from SteamSpy API using 'all' request
- Retrieve individual app data from Steam API, by iterating through app list
- Export app list, Steam data and SteamSpy data to csv files

## API references:

- https://partner.steamgames.com/doc/webapi
- https://wiki.teamfortress.com/wiki/User:RJackson/StorefrontAPI
- https://steamapi.xpaw.me/#
- https://steamspy.com/api.php


## Credits

The most important source I found while looking how to connect to the API was Nik Davis, check his blog for a different analysis on steam data (from 2019) http://nik-davis.github.io
Download functions for the APIs are based on his notebook for "Steam Data Download". I had to make some changes and simplify a bit.

Steamspy seems to have changed its API, so I had to change the download method to instead download all the data by page (set of 1000 ids). The functions defined for Steam API itself still work as is. 

In [1]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
import requests

# customisations - ensure tables show all columns
pd.set_option("max_columns", 100)

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

The next function uses requests library to get JSON response from web APIs. It is based on Nik Davis previous work, and it is quite standard as (thankfully) web APIs use a standard format, and requests makes it really easy.

In [3]:
def get_request(url,parameters=None, steamspy=False):
    """Return json-formatted response of a get request using optional parameters.
    
    Parameters
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request
    
    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    try:
        response = requests.get(url=url, params=parameters)
    except SSLError as s:
        print('SSL Error:', s)
        
        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' '*10)
        
        # recursively try again
        return get_request(url, parameters, steamspy)
    
    if response:
        return response.json()
    else:
        # We do not know how many pages steamspy has... and it seems to work well, so we will use no response to stop.
        if steamspy:
            return "stop"
        else :
            # response is none usually means too many requests. Wait and try again 
            print('No response, waiting 10 seconds...')
            time.sleep(10)
            print('Retrying.')
            return get_request(url, parameters, steamspy)

## List of IDs

APPs on steam have an unique ID. The requests to Steam API (which has more information than Steam Spy) have to be made for a specific ID. This means we have to get first a list of ids.

We can do this in several ways, but this is what I decided to follow:

* Using Steam Spy API (see https://steamspy.com/api.php) to get the list of IDs and also the metadata from Steam Spy (at the same time. Unfortunately, using this method got me lots of duplicates and headaches.


* Alternatively, we could use Steam API to get a list of apps, then filter them (see https://api.steampowered.com/ISteamApps/GetAppList/v2/? or https://steamapi.xpaw.me/#IStoreService/GetAppInfo)

We will do the last alternative, and then get the information from Steam Spy for the available IDs.

We did not know how many appids were registered in steamspy at 26/1/2022 (steamspy ranks them by user average, it seems), and it seems there are 51k appids.

The code above stops if it fails using the steamspy flag, because steamspy API is quite generous and has let us download all the data with just 1s pause between requests (although in their API it says we should wait 60s for a full page request).

Instead of telling us that a new page does not exist (by returning a null, in example) it gives us a server error, so this is a way to do it. Steam just returns null in this scenario.

In [4]:
df_steam_spy = pd.read_csv("../data/download/steamspy_appid.csv")
# generate sorted app_list from steamspy data
app_list = df_steam_spy[['appid', 'name']].sort_values('appid').reset_index(drop=True)

In [5]:
df_steam_spy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51666 entries, 0 to 51665
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       51666 non-null  int64  
 1   appid            51666 non-null  int64  
 2   name             51653 non-null  object 
 3   developer        51443 non-null  object 
 4   publisher        51495 non-null  object 
 5   score_rank       44 non-null     float64
 6   positive         51666 non-null  int64  
 7   negative         51666 non-null  int64  
 8   userscore        51666 non-null  int64  
 9   owners           51666 non-null  object 
 10  average_forever  51666 non-null  int64  
 11  average_2weeks   51666 non-null  int64  
 12  median_forever   51666 non-null  int64  
 13  median_2weeks    51666 non-null  int64  
 14  price            51637 non-null  float64
 15  initialprice     51644 non-null  float64
 16  discount         51644 non-null  float64
 17  ccu         

In [6]:
df_steam_spy["appid"].duplicated().sum()

1377

By checking the duplicated by appid, we can already see that there is some issue with this approach. It would be better to get the full steam list from Steam Store, and come back to see if we can get the data individually from Steam Spy.

## Define Download Logic

This is strongly based on Nik Davis previous work, to get the info about the app IDs from Steam. Initially I prefered to focus on the analysis rather than in the acquisition phase, but I watn to modify these functions to change them from an index based approach for the update, to instead check already existing ids in the database and download only the delta.

Later, we could maybe also add a check to see if the app has been updated or not and even redownload the info from not just new IDs, but IDs that have been updated.

I will keep the original comments from Nik Davis `in quotes` to let the reader understand the process.

`Now we have the app_list dataframe, we can iterate over the app IDs and request individual app data from the servers. Here we set out our logic to retrieve and process this information, then finally store the data as a csv file.`

`Because it takes a long time to retrieve the data, it would be dangerous to attempt it all in one go as any errors or connection time-outs could cause the loss of all our data. For this reason we define a function to download and process the requests in batches, appending each batch to an external file and keeping track of the highest index written in a separate file.`

`This not only provides security, allowing us to easily restart the process if an error is encountered, but also means we can complete the download across multiple sessions.`

`Again, we provide verbose output for rows exported, batches complete, time taken and estimated time remaining.`

In [8]:
def get_app_data(app_list, start, stop, parser, pause):
    """Return list of app data generated from parser.
    
    parser : function to handle request
    """
    app_data = []
    
    # iterate through each row of app_list, confined by start and stop
    for index, appid in app_list[start:stop].iteritems():
        print('Current index: {}'.format(index), end='\r')

        # retrive app data for a row, handled by supplied parser, and append to list
        data = parser(appid)
        app_data.append(data)

        time.sleep(pause) # prevent overloading api with requests
    
    return app_data


def process_batches(parser, app_list, download_path, data_filename, index_filename,
                    columns, begin=0, end=-1, batchsize=100, pause=1):
    """Process app data in batches, writing directly to file.
    
    parser : custom function to format request
    app_list : dataframe of appid and name
    download_path : path to store data
    data_filename : filename to save app data
    index_filename : filename to store highest index written
    columns : column names for file
    
    Keyword arguments:
    
    begin : starting index (get from index_filename, default 0)
    end : index to finish (defaults to end of app_list)
    batchsize : number of apps to write in each batch (default 100)
    pause : time to wait after each api request (defualt 1)
    
    returns: none
    """
    print('Starting at index {}:\n'.format(begin))
    
    # by default, process all apps in app_list
    if end == -1:
        end = len(app_list) + 1
    
    # generate array of batch begin and end points
    batches = np.arange(begin, end, batchsize)
    batches = np.append(batches, end)
    
    apps_written = 0
    batch_times = []
    
    for i in range(len(batches) - 1):
        start_time = time.time()
        
        start = batches[i]
        stop = batches[i+1]
        
        app_data = get_app_data(app_list, start, stop, parser, pause)
        
        rel_path = os.path.join(download_path, data_filename)
        
        # writing app data to file
        with open(rel_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')
            
            for j in range(3,0,-1):
                print("\rAbout to write data, don't stop script! ({})".format(j), end='')
                time.sleep(0.5)
            
            writer.writerows(app_data)
            print('\rExported lines {}-{} to {}.'.format(start, stop-1, data_filename), end=' ')
            
        apps_written += len(app_data)
        
        idx_path = os.path.join(download_path, index_filename)
        
        # writing last index to file
        with open(idx_path, 'w') as f:
            index = stop
            print(index, file=f)
            
        # logging time taken
        end_time = time.time()
        time_taken = end_time - start_time
        
        batch_times.append(time_taken)
        mean_time = statistics.mean(batch_times)
        
        est_remaining = (len(batches) - i - 2) * mean_time
        
        remaining_td = dt.timedelta(seconds=round(est_remaining))
        time_td = dt.timedelta(seconds=round(time_taken))
        mean_td = dt.timedelta(seconds=round(mean_time))
        
        print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))
            
    print('\nProcessing batches complete. {} apps written'.format(apps_written))

The best way to use this function and still only get the newer apps, would be to instead of passing it the fully app_list, preprocess it so it only contains the "app_delta". Also we would need it to keep the final dataframe in a different file, to perform after an append to it (in case we are adding only new app_ids), or if we add some kind of updating process, a join.

`Next we define some functions to handle and prepare the external files.`

`We use reset_index for testing and demonstration, allowing us to easily reset the index in the stored file to 0, effectively restarting the entire download process.`

`We define get_index to retrieve the index from file, maintaining persistence across sessions. Every time a batch of information (app data) is written to file, we write the highest index within app_data that was retrieved. As stated, this is partially for security, ensuring that if there is an error during the download we can read the index from file and continue from the end of the last successful batch. Keeping track of the index also allows us to pause the download, continuing at a later time.`

`Finally, the prepare_data_file function readies the csv for storing the data. If the index we retrieved is 0, it means we are either starting for the first time or starting over. In either case, we want a blank csv file with only the header row to begin writing to, se we wipe the file (by opening in write mode) and write the header. Conversely, if the index is anything other than 0, it means we already have downloaded information, and can leave the csv file alone.`

In [9]:
def reset_index(download_path, index_filename):
    """Reset index in file to 0."""
    rel_path = os.path.join(download_path, index_filename)
    
    f= open(rel_path, 'w')
    f.write("0")
        

def get_index(download_path, index_filename):
    """Retrieve index from file, returning 0 if file not found."""
    try:
        rel_path = os.path.join(download_path, index_filename)

        with open(rel_path, 'r') as f:
            index = int(f.readline())
            #This just reads the initial line
    
    except FileNotFoundError:
        index = 0
        
    return index


def prepare_data_file(download_path, filename, index, columns):
    """Create file and write headers if index is 0."""
    if index == 0:
        rel_path = os.path.join(download_path, filename)

        with open(rel_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            writer.writeheader()

## Download Steam Data

`Now we are ready to start downloading data and writing to file. We define our logic particular to handling the steam API - in fact if no data is returned we return just the name and appid - then begin setting some parameters. We define the files we will write our data and index to, and the columns for the csv file. The API doesn't return every column for every app, so it is best to explicitly set these.`

`Next we run our functions to set up the files, and make a call to process_batches to begin the process. Some additional parameters have been added for demonstration, to constrain the download to just a few rows and smaller batches. Removing these would allow the entire download process to be repeated.`

I retouched many of these parameters just to check if the download could made in batches (requesting several steamapps at the same time), or even putting a faster polling rate (right now it is one second).

The storefront API (http://store.steampowered.com/api/) is very much undocumented, but the key getaway is that is not possible. The official SteamWorks API lets us do other things and it is quite well documented, but we cannot get the data available at a steam webpage, which are the things interesting for us.

The storefront API is only accessible with these [requests](https://wiki.teamfortress.com/wiki/User:RJackson/StorefrontAPI), and according to this [stackoverflow discussion](https://stackoverflow.com/questions/46330864/steam-api-all-games):
`There is a general API rate limit for each unique IP adress of 200 requests in five minutes which is one request every 1.5 seconds.` This matches our experience, Nik Davis put a pause between requests of just 1 second, and with this we get some but a few reconnect errors. If we put no pause at all, at the end we are limited by the 200 requests every 5 minutes.

That means that for a volume of around 50k at January 2022 (the steam apps available also at steam spy, already filtered by game and some owner data...) this download will take around 21 hours. Thankfully we can resume it and do it in several batches.

If we were to build a web app and we wanted to update the information daily, we could instead try pulling the full applist from steam along with the "last updated" and only request the full appid information for those ids. This is probably what Steam Spy does, and SteamDB instead uses a more sofisticated approach by being notified of any changes to appids via steamworks.

In any case, for a one shot analysis (and not a web page where the user could explore the information), the full download approach is fine.

The goal of the next two functions is getting the full list of apps from Steam, and also getting from our already downloaded data (if any) which are the appids we already have, to get the "delta" so we only have to download the new app ids.

We could retouch them a bit, as they have a "last updated" key, so we also update new information and not just new ids. But that would be more suitable for a live webpage updated once a day, not for a full on analysis like we are presenting.

In [10]:
def getAppListBatch(url, parameters):
    json_data = get_request(url, parameters=parameters)
    steam_id = pd.DataFrame.from_dict(json_data["response"]["apps"])
    try:
        more_results = json_data["response"]["have_more_results"]
        last_appid =  json_data["response"]["last_appid"]
    except:
        more_results = False
        last_appid = False
    return more_results, steam_id, last_appid

def get_update_ids_old(updatedlist, oldlist):
    updatedlist['key1'] = 1
    oldlist['key2'] = 1
    updatedlist = pd.merge(updatedlist, oldlist, right_on=['steam_appid','name'],left_on=['appid','name'], how = 'outer')
    updatedlist = updatedlist[~(updatedlist.key2 == updatedlist.key1)]
    updatedlist = updatedlist.drop(['key1','key2','steam_appid'], axis=1)
    return updatedlist

def get_update_ids(idList, oldFullList):
    #We are going to forget about names and only care about IDs.
    idList = idList["appid"]
    oldFullList = oldFullList["steam_appid"]
    oldFullList.columns = ["appid"]
    updatedList = idList.append(oldFullList)
    updatedList = updatedList.drop_duplicates(keep=False)
    updatedList = updatedList.reset_index(drop=True)
    return updatedList


In [11]:
def getAppList():
    with open('../data/steam_key.txt') as f:
        key = f.read()

    url = "https://api.steampowered.com/IStoreService/GetAppList/v1/?"
    parameters = {"key": key}
    more_results = True
    begin = True
    # from the request we get the more_results flag and also the last_appid, so we use them for the next requests.
    while (more_results):
        more_results, steam_ids, last_appid = getAppListBatch(url, parameters)
        parameters["last_appid"] = last_appid
        if (begin):
            steam_allids = steam_ids
            begin = False
        else:
            steam_allids = steam_allids.append(steam_ids)
    return steam_allids
# request 'all' from steam spy and parse into dataframe

In [13]:
def parse_steam_request(appid):
    """Unique parser to handle data from Steam Store API.
    
    Returns : json formatted data (dict-like)
    """
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": appid, "key":"CFDD73A08F5BD5ACD3197173F81E5563"}
    
    json_data = get_request(url, parameters=parameters)
    json_app_data = json_data[str(appid)]
    
    if json_app_data['success']:
        data = json_app_data['data']
    else:
        data = {'steam_appid': appid}
        
    return data


# Set file parameters
download_path = '../data/download/'
steam_app_data = 'steam_app_data.csv'
steam_app_data_delta = 'steam_app_data_delta.csv'
steam_index = 'steam_index.txt'

steam_columns = [
    'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
    'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
    'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
    'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
    'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
    'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
    'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
    'background', 'content_descriptors'
]

# Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
if (os.path.isfile(download_path+steam_app_data_delta) == False):
    reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if no previous  data
if (os.path.isfile(download_path+steam_app_data) == False):
    prepare_data_file(download_path, steam_app_data, index, steam_columns)
    
# Wipe or create data file delta and write headers if index is 0
if (os.path.isfile(download_path+steam_app_data_delta) == False):
    prepare_data_file(download_path, steam_app_data_delta, index, steam_columns)
    
    
# Here we get the list of appids from steam
full_steam_ids = getAppList()

# Here we get the real list of ids not yet in our dataframe. If this is the first time we are downloading the data, we can skip
# This step and instead use the full app_list.
try:
    oldlist = pd.read_csv('../data/download/steam_app_data.csv', usecols = ['name','steam_appid'])
    steam_ids = get_update_ids(full_steam_ids, oldlist)
except FileNotFoundError:
    print("Pre-existing file not found. First time downloading full app data from steam. This will take a while.\n")
    steam_ids = full_steam_ids



In [42]:
print("New IDs detected: "+str(len(steam_ids)))

New IDs detected: 20247


In [43]:
# I separated the long process to be able to debug it better.
# Set end and chunksize for demonstration - remove to run through entire app list
# Here by default we passed "app_list" that contained all the information and saved it, now we will modify it a bit
# And add pre-processing and post-processing
print("Adding "+str(len(steam_ids))+" new ids.\n")
process_batches(
    parser=parse_steam_request,
    app_list=steam_ids,
    download_path=download_path,
    data_filename=steam_app_data_delta,
    index_filename=steam_index,
    columns=steam_columns,
    begin=index,
    #end=10,
    #batchsize=5
)

try:
    oldlist = pd.read_csv('../data/download/steam_app_data.csv')
    # We change the old file to backup, so remove any backup named this way before...
    os.rename('../data/download/steam_app_data.csv', '../data/download/steam_app_data_backup.csv')
    newlist = pd.read_csv('../data/download/steam_app_data_delta.csv')
    oldlist = oldlist.append(newlist, ignore_index=True)
    oldlist.to_csv('../data/download/steam_app_data.csv', index=False)
except FileNotFoundError:
    os.rename('../data/download/steam_app_data_delta.csv', '../data/download/steam_app_data.csv')

Adding 20247 new ids.

Starting at index 0:

Exported lines 0-99 to steam_app_data_delta.csv. Batch 0 time: 0:02:29 (avg: 0:02:29, remaining: 8:20:06)
Exported lines 100-199 to steam_app_data_delta.csv. Batch 1 time: 0:02:30 (avg: 0:02:29, remaining: 8:19:53)
No response, waiting 10 seconds...
Retrying.
Exported lines 200-299 to steam_app_data_delta.csv. Batch 2 time: 0:02:39 (avg: 0:02:33, remaining: 8:28:39)
Exported lines 300-399 to steam_app_data_delta.csv. Batch 3 time: 0:02:30 (avg: 0:02:32, remaining: 8:24:09)
Exported lines 400-499 to steam_app_data_delta.csv. Batch 4 time: 0:02:28 (avg: 0:02:31, remaining: 8:19:03)
Exported lines 500-599 to steam_app_data_delta.csv. Batch 5 time: 0:02:28 (avg: 0:02:31, remaining: 8:14:47)
No response, waiting 10 seconds...
Retrying.
Exported lines 600-699 to steam_app_data_delta.csv. Batch 6 time: 0:02:39 (avg: 0:02:32, remaining: 8:16:10)
Exported lines 700-799 to steam_app_data_delta.csv. Batch 7 time: 0:02:30 (avg: 0:02:32, remaining: 8:12:

Exported lines 6200-6299 to steam_app_data_delta.csv. Batch 62 time: 0:02:31 (avg: 0:02:32, remaining: 5:54:53)
Exported lines 6300-6399 to steam_app_data_delta.csv. Batch 63 time: 0:02:29 (avg: 0:02:32, remaining: 5:52:15)
Exported lines 6400-6499 to steam_app_data_delta.csv. Batch 64 time: 0:02:30 (avg: 0:02:32, remaining: 5:49:38)
No response, waiting 10 seconds...
Retrying.
Exported lines 6500-6599 to steam_app_data_delta.csv. Batch 65 time: 0:02:38 (avg: 0:02:32, remaining: 5:47:18)
Exported lines 6600-6699 to steam_app_data_delta.csv. Batch 66 time: 0:02:26 (avg: 0:02:32, remaining: 5:44:34)
No response, waiting 10 seconds...
Retrying.
Exported lines 6700-6799 to steam_app_data_delta.csv. Batch 67 time: 0:02:40 (avg: 0:02:32, remaining: 5:42:18)
Exported lines 6800-6899 to steam_app_data_delta.csv. Batch 68 time: 0:02:27 (avg: 0:02:32, remaining: 5:39:37)
No response, waiting 10 seconds...
Retrying.
Exported lines 6900-6999 to steam_app_data_delta.csv. Batch 69 time: 0:02:40 (avg

No response, waiting 10 seconds...
Retrying.
Exported lines 12500-12599 to steam_app_data_delta.csv. Batch 125 time: 0:02:40 (avg: 0:02:32, remaining: 3:15:31)
Exported lines 12600-12699 to steam_app_data_delta.csv. Batch 126 time: 0:02:27 (avg: 0:02:32, remaining: 3:12:56)
No response, waiting 10 seconds...
Retrying.
Exported lines 12700-12799 to steam_app_data_delta.csv. Batch 127 time: 0:02:40 (avg: 0:02:32, remaining: 3:10:28)
Exported lines 12800-12899 to steam_app_data_delta.csv. Batch 128 time: 0:02:27 (avg: 0:02:32, remaining: 3:07:52)
No response, waiting 10 seconds...
Retrying.
Exported lines 12900-12999 to steam_app_data_delta.csv. Batch 129 time: 0:02:38 (avg: 0:02:32, remaining: 3:05:23)
Exported lines 13000-13099 to steam_app_data_delta.csv. Batch 130 time: 0:02:30 (avg: 0:02:32, remaining: 3:02:50)
No response, waiting 10 seconds...
Retrying.
Exported lines 13100-13199 to steam_app_data_delta.csv. Batch 131 time: 0:02:37 (avg: 0:02:32, remaining: 3:00:20)
Exported lines 

No response, waiting 10 seconds...
Retrying.
Exported lines 18500-18599 to steam_app_data_delta.csv. Batch 185 time: 0:02:39 (avg: 0:02:33, remaining: 0:43:14)
Exported lines 18600-18699 to steam_app_data_delta.csv. Batch 186 time: 0:02:29 (avg: 0:02:33, remaining: 0:40:41)
No response, waiting 10 seconds...
Retrying.
Exported lines 18700-18799 to steam_app_data_delta.csv. Batch 187 time: 0:02:39 (avg: 0:02:33, remaining: 0:38:09)
Exported lines 18800-18899 to steam_app_data_delta.csv. Batch 188 time: 0:02:29 (avg: 0:02:33, remaining: 0:35:36)
No response, waiting 10 seconds...
Retrying.
Exported lines 18900-18999 to steam_app_data_delta.csv. Batch 189 time: 0:02:36 (avg: 0:02:33, remaining: 0:33:04)
Exported lines 19000-19099 to steam_app_data_delta.csv. Batch 190 time: 0:02:26 (avg: 0:02:33, remaining: 0:30:31)
No response, waiting 10 seconds...
Retrying.
Exported lines 19100-19199 to steam_app_data_delta.csv. Batch 191 time: 0:02:38 (avg: 0:02:33, remaining: 0:27:59)
Exported lines 

TypeError: read_csv() got an unexpected keyword argument 'index'

Let's ensure we have no duplicate ids and that we got them all!

In [ ]:
steam_app_data = pd.read_csv('../data/download/steam_app_data.csv')

In [48]:
steam_app_data.duplicated(subset="steam_appid").sum()

2957

We got some duplicates here. Let's compare that to the full set of ids.

In [50]:
full_steam_ids.duplicated(subset="appid").sum()

0

Even though we cleaned before, it is possible we got some ids twice. Let's compare the size.

In [54]:
len(steam_app_data)-len(full_steam_ids)

5886

Since we were using old data, it seems like ids that are no longer available are still there, along with a few duplicates. Let's run again the function which would get new ids, just to make sure.

In [55]:
diff_ids = get_update_ids(full_steam_ids, steam_app_data)

In [56]:
len(diff_ids)

14

There are only 14 new ids. Taking into account that around a hundred apps get uploaded to Steam everyday, this makes sense, so we do not need to download anything new for the moment, just cleaning.

Well, from Steam... now we have to make sure we got most of these IDs from SteamSpy as well.

Let's do a bit of pre-cleaning, to ensure we download only the ids we need from Steam Spy.

We are going to consider valid apps those that at least have a name for the moment. Then delete the remaining duplicates.

In [63]:
steam_app_data = steam_app_data[~steam_app_data["name"].isna()]
steam_app_data = steam_app_data.drop_duplicates(subset="steam_appid", keep="last")
steam_app_data.to_csv("../data/download/steam_app_data.csv", index=False)

In [64]:
# inspect downloaded data
steam_app_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66803 entries, 0 to 69853
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   type                     66655 non-null  object 
 1   name                     66803 non-null  object 
 2   steam_appid              66803 non-null  int64  
 3   required_age             66655 non-null  object 
 4   is_free                  66655 non-null  object 
 5   controller_support       14422 non-null  object 
 6   dlc                      9342 non-null   object 
 7   detailed_description     66587 non-null  object 
 8   about_the_game           66585 non-null  object 
 9   short_description        66593 non-null  object 
 10  fullgame                 0 non-null      float64
 11  supported_languages      66599 non-null  object 
 12  header_image             66655 non-null  object 
 13  website                  36119 non-null  object 
 14  pc_requirements       

## Steam Spy API

APPs on steam have an unique ID. The requests to Steam API (which has more information than Steam Spy) have to be made for a specific ID. This means we have to get first a list of ids.

We can do this in several ways, but this is what I decided to follow:

* Using Steam Spy API (see https://steamspy.com/api.php) to get the list of IDs and also the metadata from Steam Spy (at the same time). Alternatively, we could use Steam API to get a list of apps, then filter them (see https://api.steampowered.com/ISteamApps/GetAppList/v2/? or https://steamapi.xpaw.me/#IStoreService/GetAppInfo). Unfortunately, using this method got me 

* Then using Steam API to loop for each ID from the list and getting the complete info.

We are going to use this request: https://steamspy.com/api.php?request=all&page=1 - return apps 1,000-1,999 of all apps.

In [14]:
def parse_steamspy_request(appid):
    """Parser to handle SteamSpy API data."""
    url = "https://steamspy.com/api.php"
    parameters = {"request": "appdetails", "appid": appid}
    
    json_data = get_request(url, parameters)
    return json_data


# set files and columns
download_path = '../data/download'
steamspy_data = 'steamspy_data.csv'
steamspy_index = 'steamspy_index.txt'

steamspy_columns = [
    'appid', 'name', 'developer', 'publisher', 'score_rank', 'positive',
    'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
    'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
    'languages', 'genre', 'ccu', 'tags'
]

reset_index(download_path, steamspy_index)
index = get_index(download_path, steamspy_index)

# Wipe data file if index is 0
prepare_data_file(download_path, steamspy_data, index, steamspy_columns)

process_batches(
    parser=parse_steamspy_request,
    app_list=full_steam_ids["appid"],
    download_path=download_path, 
    data_filename=steamspy_data,
    index_filename=steamspy_index,
    columns=steamspy_columns,
    begin=index,
    end=len(full_steam_ids),
    batchsize=300,
    pause=0.1
)

Starting at index 0:



NameError: name 'SSLError' is not defined

Let's quickly check if we have valid data inside.

In [70]:
steam_spy_data = pd.read_csv('../data/download/steamspy_data.csv')

In [71]:
steam_spy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63968 entries, 0 to 63967
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   appid            63968 non-null  int64  
 1   name             63865 non-null  object 
 2   developer        56705 non-null  object 
 3   publisher        56740 non-null  object 
 4   score_rank       48 non-null     float64
 5   positive         63968 non-null  int64  
 6   negative         63968 non-null  int64  
 7   userscore        63968 non-null  int64  
 8   owners           63968 non-null  object 
 9   average_forever  63968 non-null  int64  
 10  average_2weeks   63968 non-null  int64  
 11  median_forever   63968 non-null  int64  
 12  median_2weeks    63968 non-null  int64  
 13  price            56854 non-null  float64
 14  initialprice     56856 non-null  float64
 15  discount         56856 non-null  float64
 16  languages        56814 non-null  object 
 17  genre       

In [72]:
steam_spy_data.duplicated(subset="appid").sum()

0

Looks good!

Now we have the Steam Spy data available on `../data/download/steamspy_data.csv` and the Steam Store data available on `../data/download/steam_app_data.csv`

## Steam Reviews

While exploring why the steam reviews on steam spy and the steam store webpage itself were not the same, we got a way to check via the API partners itself. Let's try to obtain this information.

In [39]:
def parse_steamreviews_request(appid):
    """Parser to handle SteamSpy API data."""
    url = "https://store.steampowered.com/appreviews/" + str(appid)
    parameters = {"json": 1, "num_per_page": "0", "language": "all"}
    json_data = get_request(url, parameters)
    json_data = json_data['query_summary']
    json_data["appid"]=appid
    return json_data


# set files and columns
download_path = '../data/download'
steamreviews_data = 'steamreviews_data.csv'
steamreviews_index = 'steamreviews_index.txt'

steamreviews_columns = [
    'appid', 'review_score', 'review_score_desc', 'total_positive', 'total_negative', 'total_reviews'
]

reset_index(download_path, steamreviews_index)
index = get_index(download_path, steamreviews_index)

# Wipe data file if index is 0
prepare_data_file(download_path, steamreviews_data, index, steamreviews_columns)

full_steam_ids=pd.read_csv("../data/download/steam_app_data.csv")

process_batches(
    parser=parse_steamreviews_request,
    app_list=full_steam_ids["steam_appid"],
    download_path=download_path, 
    data_filename=steamreviews_data,
    index_filename=steamreviews_index,
    columns=steamreviews_columns,
    begin=index,
    end=len(full_steam_ids),
    batchsize=300,
    pause=0
)

Starting at index 0:

Exported lines 0-299 to steamreviews_data.csv. Batch 0 time: 0:01:24 (avg: 0:01:24, remaining: 5:11:38)
Exported lines 300-599 to steamreviews_data.csv. Batch 1 time: 0:01:25 (avg: 0:01:25, remaining: 5:12:08)
Exported lines 600-899 to steamreviews_data.csv. Batch 2 time: 0:01:31 (avg: 0:01:27, remaining: 5:18:31)
Exported lines 900-1199 to steamreviews_data.csv. Batch 3 time: 0:01:32 (avg: 0:01:28, remaining: 5:21:21)
Exported lines 1200-1499 to steamreviews_data.csv. Batch 4 time: 0:01:31 (avg: 0:01:29, remaining: 5:21:41)
Exported lines 1500-1799 to steamreviews_data.csv. Batch 5 time: 0:01:30 (avg: 0:01:29, remaining: 5:20:49)
Exported lines 1800-2099 to steamreviews_data.csv. Batch 6 time: 0:01:31 (avg: 0:01:29, remaining: 5:20:29)
Exported lines 2100-2399 to steamreviews_data.csv. Batch 7 time: 0:01:31 (avg: 0:01:29, remaining: 5:19:57)
Exported lines 2400-2699 to steamreviews_data.csv. Batch 8 time: 0:01:32 (avg: 0:01:30, remaining: 5:19:34)
Exported lines 

Exported lines 44100-44399 to steamreviews_data.csv. Batch 147 time: 0:01:30 (avg: 0:01:32, remaining: 1:55:20)
Exported lines 44400-44699 to steamreviews_data.csv. Batch 148 time: 0:01:29 (avg: 0:01:32, remaining: 1:53:46)
Exported lines 44700-44999 to steamreviews_data.csv. Batch 149 time: 0:01:28 (avg: 0:01:32, remaining: 1:52:12)
Exported lines 45000-45299 to steamreviews_data.csv. Batch 150 time: 0:01:28 (avg: 0:01:32, remaining: 1:50:38)
Exported lines 45300-45599 to steamreviews_data.csv. Batch 151 time: 0:01:28 (avg: 0:01:32, remaining: 1:49:03)
Exported lines 45600-45899 to steamreviews_data.csv. Batch 152 time: 0:01:28 (avg: 0:01:32, remaining: 1:47:29)
Exported lines 45900-46199 to steamreviews_data.csv. Batch 153 time: 0:01:26 (avg: 0:01:32, remaining: 1:45:54)
Exported lines 46200-46499 to steamreviews_data.csv. Batch 154 time: 0:01:29 (avg: 0:01:32, remaining: 1:44:21)
Exported lines 46500-46799 to steamreviews_data.csv. Batch 155 time: 0:01:31 (avg: 0:01:32, remaining: 1

In [48]:
steamreviews=pd.read_csv("../data/download/steamreviews_data.csv", index_col="appid")

In [49]:
steamreviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66803 entries, 10140 to 676480
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   review_score       66803 non-null  int64 
 1   review_score_desc  66803 non-null  object
 2   total_positive     66803 non-null  int64 
 3   total_negative     66803 non-null  int64 
 4   total_reviews      66803 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.1+ MB


In [51]:
steamreviews["total_reviews"].value_counts()

0        19004
1         3938
2         3189
3         2338
4         2028
         ...  
3877         1
3813         1
12001        1
32427        1
2974         1
Name: total_reviews, Length: 3855, dtype: int64

In [52]:
steamreviews["review_score_desc"].value_counts()

No user reviews            19004
Very Positive               8178
Mixed                       7630
Positive                    6444
Mostly Positive             5331
1 user reviews              3938
2 user reviews              3189
3 user reviews              2338
4 user reviews              2028
5 user reviews              1682
6 user reviews              1438
Mostly Negative             1313
7 user reviews              1262
8 user reviews              1086
9 user reviews               888
Overwhelmingly Positive      746
Negative                     264
Very Negative                 37
Overwhelmingly Negative        7
Name: review_score_desc, dtype: int64

In [53]:
steamreviews["review_score"].value_counts()

0    36853
8     8178
5     7630
7     6444
6     5331
4     1313
9      746
3      264
2       37
1        7
Name: review_score, dtype: int64

This looks very good. Review Score Description actually gives us more information than the Score alone... although it lumps together all games with less than 10 reviews as a score of 0.

We might want to keep only the total reviews as popularity, and feature a column to have a score. But it the categories already stablished in Steam seem to be adequate. In any case, a continuous score is also good, so let's use the one stablished at steam DB: https://steamdb.info/blog/steamdb-rating/

## Next Steps

Here we have defined and demonstrated the download process used to generate the data sets. This is similar to what Nik Davis did in the past, with the exception that now the process can be reinitiated to get only the new IDs in the full id list and add them to the previous dataset. This might be expanded to get also the IDs with data updated. You might see some errors as the process had to be repeated some times, but the final raw data can be found [on Kaggle](https://www.kaggle.com/vicentearce/steam-and-steam-spy-raw-datasets).

We have two tables now with a lot of information from the apps on Stem Store. From the Steam Store API we have a lot of metadata, which is used by the Steam Store itself to display the Store page. We will consider this the main source of information. The most useful but missing information is the quantity of positive or negative recommendations, we only have the total. Also, the tags (possibly as they were added later) are not available. There might be available in a separate request which is not public, or Valve just forgot to add it to the list.

From Steam Spy we have some additional information as it tries to track the concurrent users, we have averages , top... It also offers an estimate of owners, with a very large margin of error. We will check exactly what to keep and how to clean it in the next section.

After reviewing the reviews (positive/negative reviews) vs total reviews from Steam Spy and Steam Store in the cleaning section, we discovered that they did not agree so we got a third dataset - review metadata from the Steam Store API (partners). We could have changed the request and instead get all individual reviews, which could be an interesting machine learning analysis - we have data about if they are positive or negative, and we could do a sentiment analysis from the text using NLP. But that is not the focus of our current analysis. Also we have to note that getting the metadata was about 5h, but getting all individual reviews could take quite some time.